In [51]:
import cv2
import numpy as np
from cvzone.HandTrackingModule import HandDetector
import cvzone

In [64]:
import math
import random

In [53]:
detector = HandDetector(detectionCon=0.8, maxHands=1)

In [82]:
class SnakeGameClass:
    def __init__(self, pathFood):
        self.points = [] # All point of the snake
        self.lengths = [] # Distance between each points
        self.currentLength = 0 # Total Length of the sanke
        self.allowedLength = 150 # Total allowed length
        self.previousHead = 0,0 # Previous head point
        self.score = 0 # Keep track of score
        
        self.imgFood = cv2.imread(pathFood, cv2.IMREAD_UNCHANGED)
        self.hFood, self.wFood, _ = self.imgFood.shape
        self.foodPoint = 0,0
        self.randomFoodLocation()
        
    def randomFoodLocation(self):
        self.foodPoint = random.randint(100, 850), random.randint(100, 500)
    
    def update(self, imgMain, currentHead):
        px, py = self.previousHead
        cx, cy = currentHead
        
        self.points.append([cx, cy])
        distance = math.hypot(cx-px, cy-py)
        self.lengths.append(distance)
        self.currentLength += distance
        self.previousHead = cx, cy
        
        # Length Reduction
        if self.currentLength > self.allowedLength:
            for i, length in enumerate(self.lengths):
                self.currentLength -= length
                self.lengths.pop(i)
                self.points.pop(i)
                
                if self.currentLength < self.allowedLength:
                    break
        
        # Check if snake ate the food
        rx, ry = self.foodPoint
        if rx - self.wFood // 2 < cx < rx + self.wFood // 2 and \
                    ry - self.hFood // 2 < cy < ry + self.hFood // 2:
            self.randomFoodLocation()
            self.allowedLength += 50
            self.score += 1
            print(self.score)
            
    
        # Draw snake
        if self.points:
            for i, point in enumerate(self.points):
                if i != 0:
                    cv2.line(imgMain, self.points[i-1], self.points[i], (0, 0, 255), 20)
            cv2.circle(imgMain, self.points[-1], 20, (200,0,200), cv2.FILLED)
            
        # Draw Food
        rx, ry = self.foodPoint
        imgMain = cvzone.overlayPNG(imgMain, self.imgFood, (rx - self.wFood // 2, ry - self.hFood // 2))
        
        return imgMain

In [80]:
game = SnakeGameClass("resources/Donut.png")

In [81]:
cap = cv2.VideoCapture(1)
cap.set(3, 1280)
cap.set(4, 720)

while True:
    success, img = cap.read()
    img = cv2.flip(img, 1)
    hands, img = detector.findHands(img, flipType=False)
    
    if hands:
        lmList = hands[0]['lmList']
        pointIndex = lmList[8][0:2]
        img = game.update(img, pointIndex)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
    cv2.imshow('Image', img)
    cv2.waitKey(1)
    
cap.release()
cv2.destroyAllWindows()

1
2
3
4
5
6
7
8
9
10
